In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import wandb
from wandb.keras import WandbCallback

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config =  ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
# session = InteractiveSession(config=config)

In [2]:
batch_size = 32
IMG_SIZE = 224

train_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/train'
val_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/val'
test_dir = 'D:/dataset/PASCAL_VOC_2007/VOC2007/test'

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)

val_ds = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)


test_ds = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size=batch_size
)

Found 5836 images belonging to 20 classes.
Found 1470 images belonging to 20 classes.
Found 7013 images belonging to 20 classes.


In [4]:
train_ds.class_indices

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

In [5]:
# vgg_model = tf.keras.applications.VGG16(weights = 'imagenet',
#                                         include_top = False,
#                                         input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [6]:
model = tf.keras.Sequential()

for layer in vgg_model.layers[:-1]: # just exclude last layer from copying
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False

model.add(tf.keras.layers.GlobalAveragePooling2D())
# model.add(tf.keras.layers.Dense(2048, activation='relu'))
# model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dense(len(train_ds.class_indices), activation='softmax'))
model.summary()

NameError: name 'vgg_model' is not defined

In [7]:
resnet152_model = tf.keras.applications.ResNet152(weights = 'imagenet',
                                        include_top = False,
                                        input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [8]:
output = resnet152_model.output
output = tf.keras.layers.GlobalAveragePooling2D()(output)
output = tf.keras.layers.Dense(len(train_ds.class_indices), activation='softmax')(output)

model = Model(inputs=resnet152_model.input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
wandb.init(project="2023 KICS winter", entity="yhkim", config={"learning_rate": 0.001,"epochs": 300})

model.compile(optimizer=tf.optimizers.Adam(learning_rate=wandb.config.learning_rate), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

history = model.fit(train_ds,
                    epochs=wandb.config.epochs, 
                    steps_per_epoch=len(train_ds),
                    validation_steps=len(val_ds),
                    validation_data=val_ds,
                    callbacks=[WandbCallback()])

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
c:\Users\KYH\.conda\envs\tensor\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: yhkim. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/300
183/183 [==============================] - 138s 661ms/step - loss: 2.7829 - accuracy: 0.2615 - val_loss: 2.8188 - val_accuracy: 0.2735


c:\Users\KYH\.conda\envs\tensor\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
c:\Users\KYH\.conda\envs\tensor\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: c:\Users\KYH\OneDrive - 명지대학교\DAN Lab\Research\AC-SISE\wandb\run-20230221_231934-a77185m8\files\model-best\assets


...

c:\Users\KYH\.conda\envs\tensor\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: c:\Users\KYH\OneDrive - 명지대학교\DAN Lab\Research\AC-SISE\wandb\run-20230221_231934-a77185m8\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\KYH\OneDrive - 명지대학교\DAN Lab\Research\AC-SISE\wandb\run-20230221_231934-a77185m8\files\model-best)... Done. 3.2s


Epoch 62/300
183/183 [==============================] - 99s 539ms/step - loss: 1.5022 - accuracy: 0.4825 - val_loss: 3.3350 - val_accuracy: 0.2456
Epoch 63/300
183/183 [==============================] - 99s 538ms/step - loss: 1.5061 - accuracy: 0.4784 - val_loss: 3.6514 - val_accuracy: 0.2333
Epoch 64/300
183/183 [==============================] - 99s 541ms/step - loss: 1.5006 - accuracy: 0.4769 - val_loss: 2.6495 - val_accuracy: 0.3020
Epoch 65/300
183/183 [==============================] - 99s 540ms/step - loss: 1.4673 - accuracy: 0.4878 - val_loss: 2.6559 - val_accuracy: 0.3381
Epoch 66/300
183/183 [==============================] - 99s 538ms/step - loss: 1.4435 - accuracy: 0.4940 - val_loss: 4.7611 - val_accuracy: 0.2020
Epoch 67/300
183/183 [==============================] - 99s 541ms/step - loss: 1.4483 - accuracy: 0.4978 - val_loss: 3.1573 - val_accuracy: 0.2252
Epoch 68/300
183/183 [==============================] - 100s 543ms/step - loss: 1.4339 - accuracy: 0.4997 - val_loss: 

accuracy,▁▁▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇▇▇▇█▇█████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▂▆▅▄▇▅█▄▇▆▃▇▄▃▆▅▅▁▅▄▅▆▅▅▄▅▄▆▄▄▄▅▄▆▅▅▄▄▅▄
val_loss,▂▁▁▁▁▂▁▂▁▁▄▂▃█▃▃▄▅▄▄▂▄▃▄▅▄█▅▇▃▅▃▄▄▄▃▃▄▄▅
accuracy,0.6926
best_epoch,60
best_val_loss,2.2179
epoch,299
loss,0.59425
val_accuracy,0.23265


In [ ]:
# model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), 
#               loss=tf.keras.losses.categorical_crossentropy,
#               metrics=["accuracy", "top_k_accuracy_score"])

# history = model.fit(train_ds,
#                     epochs=300, 
#                     steps_per_epoch=len(train_ds),
#                     validation_steps=len(val_ds),
#                     validation_data=val_ds)

In [10]:
model.save('resnet152_voc2007')

c:\Users\KYH\.conda\envs\tensor\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: resnet152_voc2007\assets


In [8]:
model = tf.keras.models.load_model('model/resnet50_voc2007')

In [11]:
preds = model.predict(test_ds)

In [12]:
model.evaluate(test_ds)

220/220 [==============================] - 38s 173ms/step - loss: 4.6857 - accuracy: 0.3238


[4.685732841491699, 0.32382717728614807]

In [13]:
y_true = test_ds.labels[test_ds.index_array]

In [14]:
y_true

array([ 0, 15, 14, ..., 14,  6,  6])

In [15]:
np.argmax(preds, axis=1)

array([ 2, 14,  6, ...,  6,  7,  6], dtype=int64)

In [16]:
from sklearn.metrics import top_k_accuracy_score, accuracy_score

In [17]:
top_k_accuracy_score(y_true, preds, k=5)

0.47069727648652504